## STAGING

Responsável pelo download, extração e armazenamento dos dados da fonte CEAP.

In [0]:
spark.sql("USE CATALOG mvp_ed_ceap")
spark.sql("USE SCHEMA staging")

DataFrame[]

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS mvp_ed_ceap.staging.staging_data;

In [0]:
zip_url = "https://www.camara.leg.br/cotas/Ano-2024.csv.zip"
zip_target = "dbfs:/Volumes/mvp_ed_ceap/staging/staging_data/Ano-2024.csv.zip"

dbutils.fs.cp(zip_url, zip_target)

display(dbutils.fs.ls("dbfs:/Volumes/mvp_ed_ceap/staging/staging_data/"))


path,name,size,modificationTime
dbfs:/Volumes/mvp_ed_ceap/staging/staging_data/Ano-2024.csv.zip,Ano-2024.csv.zip,7458559,1766163794000


In [0]:
from pyspark.sql.functions import col

zip_df = (
    spark.read.format("binaryFile")
    .load("dbfs:/Volumes/mvp_ed_ceap/staging/staging_data/Ano-2024.csv.zip")
)

In [0]:
import zipfile
import io

zip_bytes = zip_df.select("content").first()[0]
zip_stream = zipfile.ZipFile(io.BytesIO(zip_bytes))

# descobrir o nome real do CSV dentro do zip
csv_name = [n for n in zip_stream.namelist() if n.lower().endswith(".csv")][0]
csv_bytes = zip_stream.read(csv_name)

csv_target = "dbfs:/Volumes/mvp_ed_ceap/staging/staging_data/Ano-2024.csv"

dbutils.fs.put(csv_target, csv_bytes.decode("utf-8"), overwrite=True)

print("CSV extraído para:", csv_target)


Wrote 78460148 bytes.
CSV extraído para: dbfs:/Volumes/mvp_ed_ceap/staging/staging_data/Ano-2024.csv


In [0]:
display(dbutils.fs.ls("dbfs:/Volumes/mvp_ed_ceap/staging/staging_data/"))

path,name,size,modificationTime
dbfs:/Volumes/mvp_ed_ceap/staging/staging_data/Ano-2024.csv,Ano-2024.csv,78460148,1766163799000
dbfs:/Volumes/mvp_ed_ceap/staging/staging_data/Ano-2024.csv.zip,Ano-2024.csv.zip,7458559,1766163794000


In [0]:
df = spark.read.option("header", True).csv(
    "/Volumes/mvp_ed_ceap/staging/staging_data/Ano-2024.csv"
)
display(df.limit(5))

"""txNomeParlamentar"";""cpf"";""ideCadastro"";""nuCarteiraParlamentar"";""nuLegislatura"";""sgUF"";""sgPartido"";""codLegislatura"";""numSubCota"";""txtDescricao"";""numEspecificacaoSubCota"";""txtDescricaoEspecificacao"";""txtFornecedor"";""txtCNPJCPF"";""txtNumero"";""indTipoDocumento"";""datEmissao"";""vlrDocumento"";""vlrGlosa"";""vlrLiquido"";""numMes"";""numAno"";""numParcela"";""txtPassageiro"";""txtTrecho"";""numLote"";""numRessarcimento"";""datPagamentoRestituicao"";""vlrRestituicao"";""nuDeputadoId"";""ideDocumento"";""urlDocumento"""
"""LID.GOV-CD"";"""";"""";"""";""2023"";""NA"";"""";""57"";""1"";""MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR"";""0"";"""";""AMORETTO CAFES EXPRESSO LTDA"";""085.324.290/0013-1"";""1021"";""0"";""2024-02-07T00:00:00"";""1148.7"";""0"";""1148.7"";""2"";""2024"";""0"";"""";"""";""2018065"";"""";"""";"""";""2812"";""7696122"";""https://www.camara.leg.br/cota-parlamentar/documentos/publ/2812/2024/7696122.pdf"""
"""LID.GOV-CD"";"""";"""";"""";""2023"";""NA"";"""";""57"";""1"";""MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR"";""0"";"""";""AMORETTO CAFES EXPRESSO LTDA"";""085.324.290/0013-1"";""1106"";""0"";""2024-03-13T00:00:00"";""1148.7"";""0"";""1148.7"";""3"";""2024"";""0"";"""";"""";""2028853"";"""";"""";"""";""2812"";""7716118"";""https://www.camara.leg.br/cota-parlamentar/documentos/publ/2812/2024/7716118.pdf"""
"""LID.GOV-CD"";"""";"""";"""";""2023"";""NA"";"""";""57"";""1"";""MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR"";""0"";"""";""AMORETTO CAFES EXPRESSO LTDA"";""085.324.290/0013-1"";""1192"";""0"";""2024-04-05T00:00:00"";""1148.7"";""0"";""1148.7"";""4"";""2024"";""0"";"""";"""";""2032624"";"""";"""";"""";""2812"";""7723265"";""https://www.camara.leg.br/cota-parlamentar/documentos/publ/2812/2024/7723265.pdf"""
"""LID.GOV-CD"";"""";"""";"""";""2023"";""NA"";"""";""57"";""1"";""MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR"";""0"";"""";""AMORETTO CAFES EXPRESSO LTDA"";""085.324.290/0013-1"";""1280"";""0"";""2024-05-09T00:00:00"";""1148.7"";""0"";""1148.7"";""5"";""2024"";""0"";"""";"""";""2044731"";"""";"""";"""";""2812"";""7745812"";""https://www.camara.leg.br/cota-parlamentar/documentos/publ/2812/2024/7745812.pdf"""
"""LID.GOV-CD"";"""";"""";"""";""2023"";""NA"";"""";""57"";""1"";""MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR"";""0"";"""";""AMORETTO CAFES EXPRESSO LTDA"";""085.324.290/0013-1"";""1368"";""0"";""2024-06-19T00:00:00"";""1148.7"";""0"";""1148.7"";""6"";""2024"";""0"";"""";"""";""2057199"";"""";"""";"""";""2812"";""7769537"";""https://www.camara.leg.br/cota-parlamentar/documentos/publ/2812/2024/7769537.pdf"""
